## Microplastics AI

In [6]:
import cv2
import numpy as np
from skimage import measure
from scipy import ndimage

# Функция для обнаружения и анализа частиц микропластика
def analyze_microplastic(image_path):
    # Загрузка изображения
    img = cv2.imread(image_path)

    # Преобразование изображения в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Бинаризация изображения
    _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    # Избавление от шумов
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))

    # Нахождение контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Создание маски для частиц
    particle_mask = np.zeros_like(gray)

    # Параметры для анализа частиц
    min_particle_area = 50  # Минимальная площадь частицы
    max_particle_area = 5000  # Максимальная площадь частицы

    # Анализ каждого контура
    for contour in contours:
        area = cv2.contourArea(contour)

        if min_particle_area < area < max_particle_area:
            cv2.drawContours(particle_mask, [contour], -1, 255, thickness=cv2.FILLED)

            # Извлечение свойств частицы (размер, центр, геометрическая категория и т.д.)
            moments = cv2.moments(contour)
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])
            
            # Получение вырезанного изображения частицы
            particle_image = img[cy-50:cy+50, cx-50:cx+50]  # Пример: вырезать 100x100 область вокруг центра частицы

            # Здесь можно добавить код для определения цвета частицы и других характеристик

    return particle_mask

# Пример использования
input_image_path = "путь_к_вашей_фотографии.jpg"
particle_mask = analyze_microplastic(input_image_path)

# Сохранение результата
cv2.imwrite("particle_mask.jpg", particle_mask)

In [1]:
import cv2
import numpy as np

def analyze_microplastic(image_path):
    # Загрузка изображения
    img = cv2.imread(image_path)
    
    # Преобразование изображения в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Бинаризация изображения
    _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    
    # Избавление от шумов с помощью морфологической операции открытия
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    
    # Нахождение контуров на бинарном изображении
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Проход по каждому контуру
    for contour in contours:
        area = cv2.contourArea(contour)

        # Определение условий для выделения частиц микропластика
        if 50 < area < 5000:
            # Отрисовка контура на исходном изображении
            cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)

            # Извлечение свойств частицы
            moments = cv2.moments(contour)
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])

            # Размер частицы (площадь)
            size = area  

            # Форма частицы (в данном случае, всегда "Unknown" из-за простоты)
            shape = "Unknown"

            # Цвет частицы (определение категории цвета на основе среднего цвета внутри контура)
            mean_color = cv2.mean(img, mask=binary)[::-1]  # OpenCV возвращает BGR, переворачиваем в RGB
            color_category = determine_color_category(mean_color)

            # Вырезание изображения частицы для дальнейшего анализа
            particle_image = img[cy-50:cy+50, cx-50:cx+50]  # Пример: вырезать 100x100 область вокруг центра частицы

            # Вывод результатов
            print(f"Size: {size} pixels")
            print(f"Shape: {shape}")
            print(f"Color: {color_category}")
            
            # Дополнительные шаги для сохранения вырезанного изображения и других данных

    # Отображение исходного изображения с выделенными частицами
    cv2.imshow("Detected Microplastic", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def determine_color_category(mean_color):
    # Пример: определение категории цвета на основе среднего цвета
    # Можно дополнительно настроить для определения цвета на основе RGB-значений
    if mean_color[0] > 200 and mean_color[1] < 50 and mean_color[2] < 50:
        return "Red"
    elif mean_color[0] < 50 and mean_color[1] > 200 and mean_color[2] < 50:
        return "Green"
    elif mean_color[0] < 50 and mean_color[1] < 50 and mean_color[2] > 200:
        return "Blue"
    else:
        return "Unknown"

# Пример использования
input_image_path = "\\train\\1_jpg.rf.3bd160570ca04ce20d506e7d30bba9db.jpg"
analyze_microplastic(input_image_path)

ModuleNotFoundError: No module named 'cv2'